In [1]:
import sys
if not './' in sys.path:
    sys.path.append('./')

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp

# from tensorflow.keras.mixed_precision import experimental as mixed_precision
import numpy as np
from datetime import datetime

from envs.stocks_env_multiaction import Stocks_env
from datasets import nyse
import io

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

<h2>Load Data</h2>

In [3]:
#data = nyse.load_data('../data/')
data, tokenized_industry, vocabulary_size = nyse.load_data_with_industry('../data/')

<h2>Training Parameters</h2>

In [4]:
window_size = 5
run_lenght = 30

# Hyper params:
initial_money    = 100
batch_size       = 256
num_epochs       = 100

# log
save_directory = 'results/saved-timesteps/'
date = datetime.now().strftime("%Y_%m_%d-%H:%M:%S")
identifier = "stonks-random-" + date

<h2>Create Enviroment</h2>

In [5]:
# initialize env
env = Stocks_env(data, batch_size, window_size, run_lenght, train_test_ratio=1,
                 tokenized_industry=tokenized_industry, test_seed=None, initial_money=initial_money)

<h2>Create Model</h2>

In [6]:
# initialize the model
model = tfp.distributions.Uniform(low=np.full((batch_size,2), 0.), high=1.)

<h2>Train</h2>

In [7]:
def plot(epochs, rewards):
    clear_output(True)
    plt.figure(figsize=(20, 5))
    plt.suptitle('frame %s. ' % epochs[-1])
    plt.title('reward: %s' % rewards[-1])
    plt.plot(epochs, rewards)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    plt.show()
    return image

In [8]:
epoch  = 1
sum_profit = 0

while epoch <= num_epochs:

    env.reset(training=False, batch_size=batch_size, run_lenght=run_lenght, initial_money=initial_money)

    done = False
    total_profit = 0
    while not done:
        dist = model

        action = dist.sample()
        _, _, done, _, _, _, profit = env.step(action)
        total_profit += profit
    total_profit = sum(profit/initial_money)/batch_size
    sum_profit += total_profit 
    
    print(sum_profit/epoch)
    
    epoch += 1    
    


-0.00021800434914202085
-0.0002808055578383605
-8.460464990639773e-05
0.00016453122770272938
3.007527489187914e-05
0.00019848120656538827
0.00040335644994774985
0.0004623689501959769
0.0004692976637934081
0.00025976344217109595
0.00039013141682142714
0.0003566348290591531
0.000349389105961105
0.0003241857628700326
0.0003310824101535962
0.0003799982389068264
0.00032999059122627833
0.00040612595823507876
0.00043025999454724035
0.00038637996242918643
0.00042753702774788187
0.0005124627712431048
0.0004899616695253374
0.00046244343131980614
0.0004594991299201211
0.0004534721170044907
0.0004837461230872645
0.0005160560627396804
0.0005148500765428442
0.0005186734813514373
0.0004989403156341209
0.00048096074174970414
0.0004857081035088197
0.00046620060665920605
0.0004891189611294331
0.00045744440546053574
0.00047849653476343706
0.0004694007264837596
0.0004936968949308275
0.0004698640535140475
0.0004904619086340292
0.00046203660301475947
0.0004736318579373567
0.00046805498738577164
0.0004751529

KeyboardInterrupt: 